In [132]:
%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
import spacy
from scipy.stats import rankdata, hmean, norm
import spacy, nltk
import os, pkgutil, json, urllib
nltk.download('stopwords')
from nltk.corpus import stopwords
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from nltk import word_tokenize
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("&lt;style>.container { width:98% !important; }&lt;/style>"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giorgiomondauto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [133]:
dati_aprile = pd.read_csv('Job_vacancies_aprile_completo.csv', skiprows=1, usecols = [1,2,3],\
                          names = ['Target','Sub_Role','Job_Description'])
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: re.sub('[^a-zA-Z]',' ',x))
dati_aprile.Target = dati_aprile.Target.apply(lambda x: x.lower())
dati_aprile.Job_Description = dati_aprile['Job_Description'].apply(lambda x: word_tokenize(x.lower()))
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if len(i) >3])
Fonti = ['randstad', 'monster', 'infojob', 'technical', 'kelly', 'services', 'italia', 'lavoropi',\
             'quanta','adeccp','spa','vimercate','temporary','openjobmetis','agenzia','adecco']
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in Fonti])
list_stopwords = list(set(stopwords.words('italian')))
dati_aprile.Job_Description = dati_aprile.Job_Description.apply(lambda x: [i for i in x if i not in list_stopwords] )
# dati_aprile['Job_Description'] = dati_aprile['Job_Description'].apply(lambda x: ' '.join(x))
dati_aprile.groupby('Target').apply(lambda x: x.Job_Description.apply(lambda x: len(''.join(x).split())).sum())

Target
camerieri di ristorante             100
commessi delle vendite al minuto    149
cuochi in alberghi e ristoranti     100
elettromeccanici                     99
statistici                          104
tecnici web                          98
dtype: int64

In [134]:
dati_aprile

,Target,Sub_Role,Job_Description
0,elettromeccanici,Manutentore elettromeccanico,"[seleziona, manutentore, inserimento, somminis..."
1,elettromeccanici,Manutentore elettromeccanico,"[filiale, verona, seleziona, azienda, cliente,..."
2,elettromeccanici,Tecnico elettromeccanico,"[multinazionale, americana, leader, risorse, u..."
3,elettromeccanici,Manutentore elettromeccanico,"[everywhere, divisione, specializzata, ricerca..."
4,elettromeccanici,AGGIUSTATORE MECCANICO,"[filiale, foligno, ricerca, importante, aziend..."
...,...,...,...
645,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,"[general, proprietaria, marchio, bacci, profum..."
646,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,"[punto, vendita, rivoli, insegna, beauty, star..."
647,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,"[filiale, brescia, ricerca, addetto, vendita, ..."
648,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,"[risorse, dovranno, svolgere, attivita, vendit..."


In [135]:
geo_data = pd.read_fwf('data/listacomuni.txt')
geo_data.to_csv('data/listacomuni.csv')
geo_data = pd.read_csv('data/listacomuni.csv', sep=';')

comune = [i.lower() for i in geo_data.Comune]
geo_data.Provincia = geo_data.Provincia.apply(lambda x: str(x))
provincia = [i.lower() for i in geo_data.Provincia]
regione = [i.lower() for i in geo_data.Regione]  + ['lazio'] + ['dovr']


In [122]:
# interesse = ['tecnici web','elettromeccanici'] 
# filter_data = dati_aprile[dati_aprile.Target.isin(interesse)]
# filter_data

,Target,Sub_Role,Job_Description
0,elettromeccanici,Manutentore elettromeccanico,"[seleziona, manutentore, inserimento, somminis..."
1,elettromeccanici,Manutentore elettromeccanico,"[filiale, verona, seleziona, azienda, cliente,..."
2,elettromeccanici,Tecnico elettromeccanico,"[multinazionale, americana, leader, risorse, u..."
3,elettromeccanici,Manutentore elettromeccanico,"[everywhere, divisione, specializzata, ricerca..."
4,elettromeccanici,AGGIUSTATORE MECCANICO,"[filiale, foligno, ricerca, importante, aziend..."
...,...,...,...
396,tecnici web,Gestore di strumenti social aziendali,"[content, manager, creative, adviva, padova, v..."
397,tecnici web,Tecnici web,"[tecnico, informatico, grafica, gruppo, ingegn..."
398,tecnici web,esperta di contenuti web,"[redattore, maggioli, rimini, emilia, romagna,..."
399,tecnici web,programmatore web,"[programmatore, sviluppatore, senior, front, b..."


In [136]:
def remove_comuni(data, column, geo_data):
    ''' function to remove geographic noisy'''
    data[column] = data[column].apply(lambda x: [i for i in x if i not in comune])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in provincia])
    data[column] = data[column].apply(lambda x: [i for i in x if i not in regione])
    
    return data

filter_data = remove_comuni(dati_aprile, 'Job_Description', geo_data)

In [137]:
filter_data

,Target,Sub_Role,Job_Description
0,elettromeccanici,Manutentore elettromeccanico,"[seleziona, manutentore, inserimento, somminis..."
1,elettromeccanici,Manutentore elettromeccanico,"[filiale, seleziona, azienda, cliente, settore..."
2,elettromeccanici,Tecnico elettromeccanico,"[multinazionale, americana, leader, risorse, u..."
3,elettromeccanici,Manutentore elettromeccanico,"[everywhere, divisione, specializzata, ricerca..."
4,elettromeccanici,AGGIUSTATORE MECCANICO,"[filiale, ricerca, importante, azienda, client..."
...,...,...,...
645,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,"[general, proprietaria, marchio, bacci, profum..."
646,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,"[punto, vendita, insegna, beauty, star, ricerc..."
647,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,"[filiale, ricerca, addetto, vendita, inserire,..."
648,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,"[risorse, dovranno, svolgere, attivita, vendit..."


In [138]:
filter_data['Job_Description'] = filter_data['Job_Description'].apply(lambda x: ' '.join(x))
filter_data

,Target,Sub_Role,Job_Description
0,elettromeccanici,Manutentore elettromeccanico,seleziona manutentore inserimento somministraz...
1,elettromeccanici,Manutentore elettromeccanico,filiale seleziona azienda cliente settore meta...
2,elettromeccanici,Tecnico elettromeccanico,multinazionale americana leader risorse umane ...
3,elettromeccanici,Manutentore elettromeccanico,everywhere divisione specializzata ricerca sel...
4,elettromeccanici,AGGIUSTATORE MECCANICO,filiale ricerca importante azienda cliente agg...
...,...,...,...
645,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,general proprietaria marchio bacci profumerie ...
646,commessi delle vendite al minuto,addetto alla vendita specializzato in cosmesi ...,punto vendita insegna beauty star ricercano ad...
647,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,filiale ricerca addetto vendita inserire inter...
648,commessi delle vendite al minuto,addetto alla vendita nel settore tessile,risorse dovranno svolgere attivita vendita mag...


In [139]:
nlp = spacy.load('en')
filter_data.Job_Description = filter_data.Job_Description.apply(nlp)

In [140]:
corpus = st.CorpusFromParsedDocuments(filter_data, category_col='Target', parsed_col='Job_Description').build()

In [ ]:
camerieri di ristorante             100
commessi delle vendite al minuto    149
cuochi in alberghi e ristoranti     100
elettromeccanici                     99
statistici                          104
tecnici web 

In [154]:
html = st.produce_scattertext_explorer(corpus,
                                       category='tecnici web',
                                       category_name='tecnici Web',
                                           not_category_name='Altre Categorie',
                                       minimum_term_frequency=15,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'tecniciweb15.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [29]:
# html = produce_scattertext_explorer(corpus,
#                                     category='statistici',
#                                     category_name='Statistici',
#                                     not_category_name='cuochi',
#                                     width_in_pixels=1000,
#                                     minimum_term_frequency=5,
#                                     transform=st.Scalers.scale,
#                                     metadata=filter_data['Target'])
# file_name = 'Conventions2012ScattertextScale.html'
# open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 800, height=700)

In [130]:
ls

AlBERTo-it/
Analysis_04_2020.ipynb
Analysis_04_2020_count_vectorizer.ipynb
Bayes_Model.ipynb
Bayes_NLP.ipynb
BertApproach.ipynb
Conventions2012ScattertextLog.html
Conventions2012ScattertextRankData.html
Conventions2012ScattertextScale.html
DataVisualization.ipynb
Job_vacancies_aprile.xlsx
Job_vacancies_aprile_completo.csv
Job_vacancies_aprile_completo.xlsx
KNN/
NLP_Steps.docx
Naive.png
README.md
Role_dictionary.pickle
Role_dictionary.pkl
TecniciWeb_Statistici.html
TecniciWeb_elettromeccanici .html
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
__pycache__/
api.py
camerieri di ristorante_cuochi.html
count_vector.pkl
cuochi_commessi .html
data/
glove_WIKI
glove_WIKI.syn0.npy
info_data.csv
listacomuni.csv
main.py
model.pkl
model.py
model.sav
model_columns.pkl
models/
professioni_processing.ipynb
requirements.txt
scripts/
sentita/
statistici_cuochi.html
statistici_elettromeccanici.html
tecniciweb_elettromeccanici.html
vacancy_api.mov
~$Job_vacancies_aprile.xlsx


In [31]:
# html = produce_scattertext_explorer(corpus,
#                                     category='statistici',
#                                     category_name='Statistici',
#                                     not_category_name='Other',
#                                     width_in_pixels=1000,
#                                     minimum_term_frequency=5,
#                                     transform=st.Scalers.percentile,
#                                     metadata=dati_aprile['Target'])
# file_name = 'Conventions2012ScattertextRankData.html'
# open(file_name, 'wb').write(html.encode('utf-8'))
# IFrame(src=file_name, width = 1200, height=700)